In [1]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
from collections import defaultdict
from collections import OrderedDict
import datetime as DT
import ast
import numpy as np
from string import punctuation
import urllib.request, json 

from afinn import Afinn
af = Afinn()

In [2]:
def Quarterlookup(date):
    if date>=DT.datetime(2018,10,1):
        return '18Q4'
    elif date>=DT.datetime(2018,7,1):
        return '18Q3'
    elif date>=DT.datetime(2018,4,1):
        return '18Q2'
    elif date>=DT.datetime(2018,1,1):
        return '18Q1'
    elif date>=DT.datetime(2017,10,1):
        return '17Q4'
    elif date>=DT.datetime(2017,7,1):
        return '17Q3'
    elif date>=DT.datetime(2017,4,1):
        return '17Q2'
    elif date>=DT.datetime(2017,1,1):
        return '17Q1'
    elif date>=DT.datetime(2016,10,1):
        return '16Q4'
    elif date>=DT.datetime(2016,7,1):
        return '16Q3'
    elif date>=DT.datetime(2016,4,1):
        return '16Q2'
    elif date>=DT.datetime(2016,1,1):
        return '16Q1'
    elif date>=DT.datetime(2015,10,1):
        return '15Q4'
    elif date>=DT.datetime(2015,7,1):
        return '15Q3'
    elif date>=DT.datetime(2015,4,1):
        return '15Q2'
    else:
        return '15Q1'

In [3]:
d=defaultdict(list)
with open('InputX.txt') as f:
    for line in f:
        count=0
        temp =-1
        k,v=line.split(':',1)
        
        while True:
       
            oldtemp=temp
            temp=v.find(']]',temp+1)
            if count==0:
                d[k].append(v[oldtemp+2:temp+2]) 
            else:
                d[k].append(v[oldtemp+4:temp+2])
           
             
            count+=1
            if temp == -1: break
     
  

In [4]:
##Converting the string of data into dictionary
InputX=defaultdict(dict)
count=0

for key,value in d.items():
    Inputx=defaultdict(list)

    for value2 in value:
        
        temp=value2.find('), ')
        date=value2[14:temp+3]
        try:
            year=int(date[1:5])
            month=int(date[date.find(',')+2:date.find(',',date.find(',')+1)])
            day=int(date[date.find(')')-2:date.find(')')])
            sentiment_scoreaf=0
            sentiment_scoresn=0
            for i in ast.literal_eval(value2[temp+3:-1]):     
                sentiment_scoreaf+=af.score(i)
  
            Inputx[Quarterlookup(DT.datetime(year,month,day))].append(sentiment_scoreaf)
            count+=1
        except:
            pass
    InputX[key]=OrderedDict(sorted(Inputx.items()))


In [5]:
##List of stockname and relevant stock code
l = [("CVSHealth", "CVS"),
("RiteAid", "RAD"),
("Conn's", "CONN"),
("Casey's", "CASY"),
("Kroger", "KR"),
("WeisMarkets", "WMK"),
("HavertyFurniture", "HVT"),
("Kirkland's", "KIRK"),
("Pier1Imports","PIR"),
("BuildersFirstSource","BLDR"),
("HuttigBuildingProducts", "HBP"),
("LumberLiquidators", "LL"),
("TransWorldEntertainment", "TWMC"),
("MarineMax", "HZO"),
("Medifast", "MED"),
("SallyBeauty", "SBH"),
("Sotheby's", "BID"),
("StarGasPartners", "SGU"),
("SuburbanPropanePartners", "SPH"),
("TitanMachinery", "TITN"),
("TractorSupply","TSCO"),
("TravelCentersofAmerica", "TA"),
("Big5SportingGoods", "BGFV"),
("Dick'sSportingGoods", "DKS"),
("HibbettSports", "HIBB"),
("Build-A-BearWorkshop", "BBW")]
     
all_stock = {}
[all_stock.update({k:v}) for k,v in l]
print(all_stock)

{'CVSHealth': 'CVS', 'RiteAid': 'RAD', "Conn's": 'CONN', "Casey's": 'CASY', 'Kroger': 'KR', 'WeisMarkets': 'WMK', 'HavertyFurniture': 'HVT', "Kirkland's": 'KIRK', 'Pier1Imports': 'PIR', 'BuildersFirstSource': 'BLDR', 'HuttigBuildingProducts': 'HBP', 'LumberLiquidators': 'LL', 'TransWorldEntertainment': 'TWMC', 'MarineMax': 'HZO', 'Medifast': 'MED', 'SallyBeauty': 'SBH', "Sotheby's": 'BID', 'StarGasPartners': 'SGU', 'SuburbanPropanePartners': 'SPH', 'TitanMachinery': 'TITN', 'TractorSupply': 'TSCO', 'TravelCentersofAmerica': 'TA', 'Big5SportingGoods': 'BGFV', "Dick'sSportingGoods": 'DKS', 'HibbettSports': 'HIBB', 'Build-A-BearWorkshop': 'BBW'}


In [6]:
##Read the quarterly return detail from edgaronline and convert into library
top="http://datafied.api.edgar-online.com/v2/corefinancials/qtr?primarysymbols="
bottom="&appkey=asc97xrhkyu4959aptu76zxj"
Outputy =defaultdict(dict)
for stockname,stock in all_stock.items():
    check=top+stock+bottom
    with urllib.request.urlopen(check) as url:
        data = json.loads(url.read().decode())
    count1=0
    count2=0
    end_date=[]
    revenue_change=[]
    last_rev=0

    for key, value in data.items():
        
        for key2, value2 in value.items():          
        
            if count1!=0:  
                for i in reversed(value2):
                  
                    for j in i["values"]:
                     
                        if count2!=0:   
                            if j["field"] =="periodenddate":
                                end_date.append(Quarterlookup(DT.datetime.strptime(j['value'], '%m/%d/%Y')))
                       
                            ##Convert revenue into the y_label of 0 for negative growth and 1 for positive growth
                            if j["field"] =="totalrevenue":
                                revenue_cal =round(float(j['value'])/float(last_rev) - 1,3)
                                if revenue_cal>0:
                                    revenue_change.append(1)
                                else:
                                    revenue_change.append(0)
                             
                        else:
                            
                            if j["field"] =="totalrevenue":      
                                last_rev=j['value']
                              
                    count2+=1
            count1+=1
            
    ##Output will be a dictionary with stockname
    Outputy[stockname]["period"]=end_date
    Outputy[stockname]["revenuechange"]=revenue_change
        

In [43]:
stock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger', 'WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']
quarter=['15Q1','15Q2','15Q3','15Q4','16Q1','16Q2','16Q3','16Q4','17Q1','17Q2','17Q3','17Q4','18Q1','18Q2']

X_value=[]
Y_value=[]
for i in stock:
    for j in range(len(quarter)):
        X_value.append(np.mean(InputX[i][quarter[j]]))
        Y_value.append(Outputy[i]['revenuechange'][j])

In [44]:
teststock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger', 'WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']
testquarter=['18Q3']

X_valuetest=[]
Y_valuetest=[]
for i in teststock:
    for j in range(len(testquarter)):
        X_valuetest.append(np.mean(InputX[i][quarter[j]]))
        Y_valuetest.append(Outputy[i]['revenuechange'][j])

In [41]:
print(len(X_valuetest),len(Y_valuetest))

52 52


In [9]:
print(X_value,Y_value)

[1.1982484076433122, 1.1264181523500811, 1.176153846153846, 1.180776243813493, 1.1463288521199586, 1.2232754081095314, 1.1666223050306095, 1.1217345872518287, 1.137191854233655, 1.2099596231493943, 1.1869111280875986, 1.0936091222487403, 1.181159420289855, 1.2067024128686328, 1.1845413212088793, 3.103782432010003, 3.0683986320273595, 3.1150065142721783, 3.126822508735992, 3.1266960029336266, 3.0459086730003064, 3.069900228392836, 3.0545607499098666, 3.0800024261539396, 3.133482792050412, 3.0913003940588055, 3.0554882481221224, 3.09474203291623, 3.0611409723918888, 3.1041064547206165, 0.37250028245396, 0.410342180302698, 0.34383561643835614, 0.35275470048097946, 0.34059493530907886, 0.3860692645505587, 0.4054320987654321, 0.3763505731366204, 0.3668649012567325, 0.4230705024762117, 0.37543497172683776, 0.3559720249153098, 0.4194666517582602, 0.4206064008983717, 0.38907512227290214, 0.8135058056630678, 0.7669218372280419, 0.8119243339605824, 0.817196819085487, 0.7910990009082652, 0.793023

In [45]:
print(np.corrcoef(X_value,Y_value)[0, 1])

0.11658408514520843


In [15]:
print(len(X_value))
print(len(Y_value))


390
390


In [46]:
import statsmodels.api as sm

model = sm.OLS(Y_value, X_value)
results2 = model.fit()
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.306
Method:                 Least Squares   F-statistic:                     161.4
Date:                Sun, 02 Dec 2018   Prob (F-statistic):           7.55e-31
Time:                        08:29:00   Log-Likelihood:                -330.26
No. Observations:                 364   AIC:                             662.5
Df Residuals:                     363   BIC:                             666.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.3010      0.024     12.705      0.0

In [52]:
Right=0
Total=0
for i in range(len(X_valuetest)):
    if X_valuetest[i]>0:
        if Y_valuetest[i]==1:
            Right+=1
            Total+=1
        else:
            Total+=1
    else:
        if Y_valuetest[i]==0:
            Right+=1
            Total+=1
        else:
            Total+=1



print('Average precision score: {0:0.2f}'.format(
      Right/Total))

Average precision score: 0.46


In [51]:
from sklearn.metrics import average_precision_score

y_pred=[]
for i in X_valuetest:
    if i>0:
        y_pred.append(1)
    else:
        y_pred.append(0)
    

average_precision = average_precision_score(y_pred,Y_valuetest)

print('Average precision score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.84


In [50]:
print(X_valuetest)

[1.1982484076433122, 3.103782432010003, 0.37250028245396, 0.8135058056630678, 1.4572601717058604, 1.9060514877967234, 0.4634411524870889, 0.319352437662821, 2.1498923914419548, 0.4338241275522655, 0.3356453798931675, 2.0341789705522473, -0.39645922746781115, 1.8684244819947005, 1.0799840031993602, 2.7378362386782578, 0.18307223440089826, 0.19398928321814088, 0.0, 1.0515017667844524, 1.5823682748600487, 0.4298110677237408, -0.09711573566995253, -1.1752202376141598, 0.5580303933253874, 1.2754827983118047]
